# n-Step Bootstrapping: Off-policy Tree Backup, Stochastic

In [1]:
import numpy as np

## Create environment

In [2]:
def create_environment_states():
    """Creates environment states.

    Returns:
        num_states: int, number of states.
        num_terminal_states: int, number of terminal states.
        num_non_terminal_states: int, number of non terminal states.
    """
    num_states = 16
    num_terminal_states = 2
    num_non_terminal_states = num_states - num_terminal_states

    return num_states, num_terminal_states, num_non_terminal_states

In [3]:
def create_environment_actions(num_non_terminal_states):
    """Creates environment actions.

    Args:
        num_non_terminal_states: int, number of non terminal states.

    Returns:
        max_num_actions: int, max number of actions possible.
        num_actions_per_non_terminal_state: array[int], number of actions per
            non terminal state.
    """
    max_num_actions = 4

    num_actions_per_non_terminal_state = np.repeat(
        a=max_num_actions, repeats=num_non_terminal_states)

    return max_num_actions, num_actions_per_non_terminal_state

In [4]:
def create_environment_successor_counts(num_states, max_num_actions):
    """Creates environment successor counts.

    Args:
        num_states: int, number of states.
        max_num_actions: int, max number of actions possible.
    Returns:
        num_sp: array[int], number of successor
            states s' that can be reached from state s by taking action a.
    """
    num_sp = np.repeat(
        a=1, repeats=num_states * max_num_actions)

    num_sp = np.reshape(
        a=num_sp,
        newshape=(num_states, max_num_actions))

    return num_sp

In [5]:
def create_environment_successor_arrays(
        num_non_terminal_states, max_num_actions):
    """Creates environment successor arrays.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
    Returns:
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
    """
    sp_idx = np.array(
        object=[1, 0, 14, 4,
                2, 1, 0, 5,
                2, 2, 1, 6,
                4, 14, 3, 7,
                5, 0, 3, 8,
                6, 1, 4, 9,
                6, 2, 5, 10,
                8, 3, 7, 11,
                9, 4, 7, 12,
                10, 5, 8, 13,
                10, 6, 9, 15,
                12, 7, 11, 11,
                13, 8, 11, 12,
                15, 9, 12, 13],
        dtype=np.int64)

    p = np.repeat(
        a=1.0, repeats=num_non_terminal_states * max_num_actions * 1)

    r = np.repeat(
        a=-1.0, repeats=num_non_terminal_states * max_num_actions * 1)

    sp_idx = np.reshape(
        a=sp_idx,
        newshape=(num_non_terminal_states, max_num_actions, 1))
    p = np.reshape(
        a=p,
        newshape=(num_non_terminal_states, max_num_actions, 1))
    r = np.reshape(
        a=r,
        newshape=(num_non_terminal_states, max_num_actions, 1))

    return sp_idx, p, r

In [6]:
def create_environment():
    """Creates environment.

    Returns:
        num_states: int, number of states.
        num_terminal_states: int, number of terminal states.
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_actions_per_non_terminal_state: array[int], number of actions per
            non terminal state.
        num_sp: array[int], number of successor
            states s' that can be reached from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
    """
    (num_states,
     num_terminal_states,
     num_non_terminal_states) = create_environment_states()

    (max_num_actions,
     num_actions_per_non_terminal_state) = create_environment_actions(
        num_non_terminal_states)

    num_sp = create_environment_successor_counts(
        num_states, max_num_actions)

    (sp_idx,
     p,
     r) = create_environment_successor_arrays(
        num_non_terminal_states, max_num_actions)

    return (num_states,
            num_terminal_states,
            num_non_terminal_states,
            max_num_actions,
            num_actions_per_non_terminal_state,
            num_sp,
            sp_idx,
            p,
            r)

## Set hyperparameters

In [7]:
def set_hyperparameters():
    """Sets hyperparameters.

    Returns:
        n_steps: int, number of timesteps within value update.
        num_episodes: int, number of episodes to train over.
        maximum_episode_length: int, max number of timesteps for an episode.
        alpha: float, alpha > 0, learning rate.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
    """
    n_steps = 4
    num_episodes = 200000
    maximum_episode_length = 2000
    alpha = 0.05
    gamma = 1.0

    return n_steps, num_episodes, maximum_episode_length, alpha, gamma

## Create value function and policy arrays

In [8]:
def create_episode_log(maximum_episode_length):
    """Creates episode log.

    Args:
        maximum_episode_length: int, max number of timesteps for an episode.

    Returns:
        episode_log: dict, dictionary of state, action, and reward timestep
            history arrays for episode.
    """
    episode_log = {
        "s_idx": np.repeat(a=-1, repeats=maximum_episode_length),
        "a_idx": np.repeat(a=-1, repeats=maximum_episode_length),
        "reward": np.repeat(a=0.0, repeats=maximum_episode_length)
    }

    return episode_log

In [9]:
def create_value_function_arrays(num_states, max_num_actions):
    """Creates value function arrays.

    Args:
        num_states: int, number of states.
        max_num_actions: int, max number of actions possible.
    Returns:
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
    """
    q = np.repeat(a=0.0, repeats=num_states * max_num_actions)
    q = np.reshape(a=q, newshape=(num_states, max_num_actions))

    return q

In [10]:
def create_policy_arrays(num_non_terminal_states, max_num_actions):
    """Creates policy arrays.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
    Returns:
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    policy = np.repeat(
        a=1.0 / max_num_actions,
        repeats=num_non_terminal_states * max_num_actions)

    policy = np.reshape(
        a=policy,
        newshape=(num_non_terminal_states, max_num_actions))

    return policy

## Create algorithm

In [11]:
# Set random seed so that everything is reproducible
np.random.seed(seed = 0)

In [12]:
def initialize_epsiode(
        num_non_terminal_states,
        max_num_actions,
        policy,
        episode_log):
    """Initializes epsiode with initial state and initial action.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        episode_log: dict, dictionary of state, action, and reward timestep
            history arrays for episode.
    Returns:
        episode_log: dict, dictionary of state, action, and reward timestep
            history arrays for episode.
    """
    # Randomly choose an initial state from all non-terminal states
    episode_log["s_idx"][0] = np.random.randint(
        low=0, high=num_non_terminal_states, dtype=np.int64)

    # Get initial action
    episode_log["a_idx"][0] = np.random.choice(
        a=max_num_actions,
        p=policy[episode_log["s_idx"][0], :])

    return episode_log

In [13]:
def greedy_policy_from_state_action_function(q, s_idx, policy):
    """Create greedy policy from state-action value function.

    Args:
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        s_idx: int, current state index.
        policy: array[float], learned stochastic policy of which action a to
            take in state s.
    Returns:
        policy: array[float], learned stochastic policy of which action a to
            take in state s.
    """
    # Save max state-action value and find the number of actions that have the
    # same max state-action value
    max_action_value = np.max(a=q[s_idx, :])
    max_action_count = np.count_nonzero(a=q[s_idx, :] == max_action_value)
    
    # Apportion policy probability across ties equally for state-action pairs
    # that have the same value and zero otherwise
    max_policy_prob_per_action = 1.0 / max_action_count
    policy[s_idx, :] = np.where(
        q[s_idx, :] == max_action_value,
        max_policy_prob_per_action,
        0.0)
    
    return policy

In [14]:
def loop_through_episode(
        num_non_terminal_states,
        max_num_actions,
        num_sp,
        sp_idx,
        p,
        r,
        q,
        policy,
        alpha,
        gamma,
        maximum_episode_length,
        episode_log,
        n_steps):
    """Loops through episode to iteratively update policy.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_sp: array[int], number of successor states s' that can be reached
            from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        alpha: float, alpha > 0, learning rate.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        maximum_episode_length: int, max number of timesteps for an episode.
        episode_log: dict, dictionary of state, action, and reward timestep
            history arrays for episode.
        n_steps: int, number of timesteps within value update.
    Returns:
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    max_timestep = maximum_episode_length
    # Loop through episode steps until termination
    for t in range(0, maximum_episode_length):
        # Spend a little memory to save computation time
        t_mod_n_plus_1 = t % (n_steps + 1)
        t_plus_1_mod_n_plus_1 = (t + 1) % (n_steps + 1)

        s_idx = episode_log["s_idx"][t_mod_n_plus_1]
        a_idx = episode_log["a_idx"][t_mod_n_plus_1]

        if t < max_timestep:
            # Get reward
            sst_idx = np.random.choice(
                a=num_sp[s_idx, a_idx],
                p=p[s_idx, a_idx, :])

            reward = r[s_idx, a_idx, sst_idx]
            episode_log["reward"][t_plus_1_mod_n_plus_1] = reward

            # Get next state
            next_s_idx = sp_idx[s_idx, a_idx, sst_idx]
            episode_log["s_idx"][t_plus_1_mod_n_plus_1] = next_s_idx

            # Check to see if we actioned into a terminal state
            if next_s_idx >= num_non_terminal_states:
                max_timestep = t + 1
            else:
                # Randomly choose next action from next state
                next_a_idx = np.random.randint(
                    low=0, high=max_num_actions, dtype=np.int64)
                episode_log["a_idx"][t_plus_1_mod_n_plus_1] = next_a_idx

        # tau is the time whose estimate is being updated
        tau = t - n_steps + 1

        if tau >= 0:
            # Calculate expected return
            if t + 1 >= max_timestep:
                # Calculate expected return
                time_idx = max_timestep % (n_steps + 1)
                expected_return = episode_log["reward"][time_idx]
            else:
                # Calculate expected state value function from policy
                v_expected_value_on_policy = np.sum(
                    a=policy[next_s_idx, :] * q[next_s_idx, :])

                # Calculate expected return
                reward = episode_log["reward"][t_plus_1_mod_n_plus_1]
                expected_return = reward + gamma * v_expected_value_on_policy

            for k in range(min(t, max_timestep - 1), tau, -1):
                # Spend a little memory to save computation time
                k_mod_n_plus_1 = k % (n_steps + 1)

                # Calculate expected state value function from policy,
                # however without including kth chosen action
                not_action_taken_mask = np.arange(
                    max_num_actions) != episode_log["a_idx"][k_mod_n_plus_1]
                not_action_taken_policy = np.extract(
                    condition=not_action_taken_mask,
                    arr=policy[episode_log["s_idx"][k_mod_n_plus_1], :])
                not_action_taken_q = np.extract(
                    condition=not_action_taken_mask,
                    arr=q[episode_log["s_idx"][k_mod_n_plus_1], :])

                v_expected_value_on_policy = np.sum(
                    a=not_action_taken_policy * not_action_taken_q)

            # Spend a little memory to save computation time
            tau_mod_n_plus_1 = tau % (n_steps + 1)
            tau_s_idx = episode_log["s_idx"][tau_mod_n_plus_1]
            tau_a_idx = episode_log["a_idx"][tau_mod_n_plus_1]

            # Calculate state-action-function at tau timestep
            delta = expected_return - q[tau_s_idx, tau_a_idx]
            q[tau_s_idx, tau_a_idx] += alpha * delta

            # Choose policy for chosen state by epsilon-greedy choosing from
            # the state-action-value function
            policy = greedy_policy_from_state_action_function(
                q, tau_s_idx, policy)

        if tau == max_timestep - 1:
            break  # break episode step loop, move on to next episode

    return q, policy

In [15]:
def off_policy_n_step_bootstrapping_tree_backup(
        num_non_terminal_states,
        max_num_actions,
        num_sp,
        sp_idx,
        p,
        r,
        q,
        policy,
        alpha,
        gamma,
        num_episodes,
        maximum_episode_length,
        episode_log,
        n_steps):
    """Loops through episodes to iteratively update policy.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_sp: array[int], number of successor states s' that can be reached
            from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        alpha: float, alpha > 0, learning rate.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        num_episodes: int, number of episodes to train over.
        maximum_episode_length: int, max number of timesteps for an episode.
        episode_log: dict, dictionary of state, action, and reward timestep
            history arrays for episode.
        n_steps: int, number of timesteps within value update.
    Returns:
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    for episode in range(0, num_episodes):
        # Initialize episode to get initial state and action
        episode_log = initialize_epsiode(
            num_non_terminal_states,
            max_num_actions,
            policy,
            episode_log)

        # Loop through episode and update the policy
        q, policy = loop_through_episode(
            num_non_terminal_states,
            max_num_actions,
            num_sp,
            sp_idx,
            p,
            r,
            q,
            policy,
            alpha,
            gamma,
            maximum_episode_length,
            episode_log,
            n_steps)

    return q, policy

## Run algorithm

In [16]:
def run_algorithm():
    """Runs the algorithm."""
    (num_states,
     num_terminal_states,
     num_non_terminal_states,
     max_num_actions,
     num_actions_per_non_terminal_state,
     num_sp,
     sp_idx,
     p,
     r) = create_environment()

    (n_steps,
     num_episodes,
     maximum_episode_length,
     alpha,
     gamma) = set_hyperparameters()

    episode_log = create_episode_log(maximum_episode_length)

    q = create_value_function_arrays(num_states, max_num_actions)

    policy = create_policy_arrays(num_non_terminal_states, max_num_actions)

    # Print initial arrays
    print("\nInitial state-action value function")
    print(q)

    print("\nInitial policy")
    print(policy)

    # Run on policy n-step bootstrapping sarsa
    q, policy = off_policy_n_step_bootstrapping_tree_backup(
        num_non_terminal_states,
        max_num_actions,
        num_sp,
        sp_idx,
        p,
        r,
        q,
        policy,
        alpha,
        gamma,
        num_episodes,
        maximum_episode_length,
        episode_log,
        n_steps)

    # Print final results
    print("\nFinal state-action value function")
    print(q)

    print("\nFinal policy")
    print(policy)

In [17]:
run_algorithm()


Initial state-action value function
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Initial policy
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]

Final state-action value function
[[-2.99774574 -2.44179948 -1.         -2.48279976]
 [-3.13165697 -2.90680095 -2.31512945 -3.00587791]
 [-3.32875676 -3.37928397 -2.70049158 -3.15115737]
 [-2.9657913  -1.         -2.45394261 -3.18192262]
 [-3.11894451 -2.66316626 -2.32846724 -3.03710149]
 [-3.05239319 -3.07262312 -2.99328998 -2.73403838]
 [-3.06047893 -3.30568351 -2.98893872 -2.6